In [107]:
import numpy as np
import pandas as pd
import os
import sklearn 
import scipy
from scipy import stats
from sklearn.preprocessing import StandardScaler

In [285]:
homedir = '/Users/Lraymond/Dropbox (MIT)/Classes/Fall 2018/Nonlinear/psets/Pset3'
y = 'part'

In [310]:
df_mroz = pd.read_csv(os.path.join(homedir, 'mroz.csv'))

In [372]:
df_mroz['constant'] = 1.0

In [373]:
y_mroz = df_mroz.values[:,0]

In [374]:
X_mroz = df_mroz.values[:,1:]

In [375]:
print(X_mroz.shape)
print(y_mroz.shape)

(753, 5)
(753,)


In [556]:
class ProbitRegression:

    def __init__(self, y, X, B):
        self.X, self.y, self.B = X, y, B
        self.n, self.k = X.shape
    
    def logL(self, B=None):
        if B is None:
            B = self.B
        xb = self.X@B.T
#         F = stats.norm.cdf(xb, loc=xb.mean(), scale=xb.std())
        F = stats.norm.cdf(xb)
        print('Log likelihood', np.isinf(F).sum())
        F[F==0] = 1e-20
        r = self.y*np.log(F) - (1-self.y)*np.log(1-F)
        print(r[np.isnan(r)])
        s = r.sum(axis=0)
        if np.isnan(s):
            return F, xb , np.log(F), np.log(1-F)
    
    
    
    def gradient(self, B=None):
        '''Should produce a kx1 vector'''
        if B is None:
            B = self.B
        xb = self.X@B.T
        F = stats.norm.cdf(xb)
        f = stats.norm.pdf(xb)
#         F = stats.norm.cdf(xb, loc=xb.mean(), scale=xb.std())
#         f = stats.norm.pdf(xb, loc=xb.mean(), scale=xb.std())
        print(np.isinf(F).sum())
        print(np.isinf(f).sum())
        scalars = f*self.y*(f/F) - (1-self.y)*f/(1-F)
        print(np.isinf(scalars).sum())
        scalars[np.isinf(scalars)] = 0
#         sc = np.sum(scalars)
        return scalars.T@X_mroz
    
    def hessian(self, B=None):
        '''Should produce a kxx vector - X is nxk '''
        if B is None:
            B = self.B
#         normPDF = stats.norm(0, np.sqrt(1-h2))
#         invRegParam = 1.0/regParam 
        XBeta = self.X.dot(B)
#         phiXBeta = stats.norm.pdf(XBeta, loc=XBeta.mean(), scale=XBeta.std())
#         PhiXBeta = stats.norm.cdf(XBeta, loc=XBeta.mean(), scale=XBeta.std())
        phiXBeta = stats.norm.pdf(XBeta)
        PhiXBeta = stats.norm.cdf(XBeta)
        print(np.isinf(phiXBeta).sum())
        print(np.isinf(PhiXBeta).sum())
        R = np.zeros(self.X.shape[0])
        cases = (self.y==1)
        controls = (self.y==0)
        R[cases] = (XBeta[cases]*PhiXBeta[cases] + phiXBeta[cases]) / PhiXBeta[cases]**2
        R[controls] = (-XBeta[controls]*(1-PhiXBeta[controls]) + phiXBeta[controls]) / (1 - PhiXBeta[controls])**2
        R *= phiXBeta 
        H = (self.X.T * R).dot(self.X) 
        return H
#         xb = self.X@B.T
#         F = stats.norm.cdf(xb, loc=xb.mean(), scale=xb.std())
#         f = stats.norm.pdf(xb, loc=xb.mean(), scale=xb.std())
#         return F, f, xb
#         print(np.isinf(f).sum())
#         print(np.isinf(F).sum())

#         scalars = f*(self.y*(f + xb*F)/np.square(F)) + (1-self.y)*(f + xb*(1-F))/np.square(1-F)
# #         return F, f, xb
#         print(np.isinf(scalars).sum())
#         scalars[np.isinf(scalars)] = 0
#         sc = np.sum(scalars)
#         return -(self.X.T*z).dot(self.X)

    

In [553]:
def newton_raphson(model, tol=1e-20, max_iter=1000, display=True):
    def get_inv(mat):
        try:
            mat_inv = np.linalg.inv(mat)
        except np.linalg.LinAlgError as e:
            print(e)
            mat_inv = np.linalg.pinv(mat)
        return mat_inv
        

    i = 0
    error = 100  # Initial error value

    # Print header of output
    if display:
        print("-"*10)
    betas = model.B
    # While loop runs while any value in error is greater
    # than the tolerance until max iterations are reached
    while np.any(error > tol) and i < max_iter:
        print('Log Liklihood :', model.logL(betas))
        H, G = model.hessian(betas), model.gradient(betas)
        print('Shapes H, G: ', H.shape, G.shape)  
        H_inv = get_inv(H)  
        diff = (H_inv @ G).T
        B_new = betas - diff
        error = B_new - betas
        print('ERROR ', np.max(error))
        print('B OLD ', betas)
        print('B NEW ', B_new)
        model.B = B_new.copy()
        betas = B_new.copy()
        i += 1

    print('Number of iterations: {}'.format(i))
    print('B_hat = ', model.B)
    print('Log Lik = ', model.logL())
    

    return betas, model.hessian(betas), model.B

In [554]:
scaler = StandardScaler()
X_mroz_s = scaler.fit_transform(X_mroz)

In [557]:
#init_B = np.array([-.88, -.03, -.15, .02, .2])
init_B = np.array([0, 0, 0, 0, 0])
probit = ProbitRegression(y_mroz, X_mroz_s, B=init_B)
# xb, F, r = probit.logL()
B_hat, H, other_B = newton_raphson(probit, max_iter=10000, display=True)

----------
Log likelihood 0
[]
Log Liklihood : None
0
0
0
0
0
Shapes H, G:  (5, 5) (5,)
Singular matrix
ERROR  14.03909941588425
B OLD  [0 0 0 0 0]
B NEW  [ 5.79597853 14.03909942  2.56530639  4.5986116   0.        ]
Log likelihood 0
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
Log Liklihood : (array([4.82438538e-044, 1.75147847e-138, 1.18035869e-016, 3.34254337e-118,
       1.19282109e-022, 1.00000000e+000, 8.97930620e-036, 1.00000000e+000,
       9.99999973e-001, 2.86789593e-033, 5.58706362e-072, 1.07629032e-007,
       4.50935189e-056, 7.23022300e-006, 4.38417616e-003, 1.62261450e-074,
       2.88682476e-006, 1.58112916

/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in true_divide
/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encoun

In [539]:
(probit.y*np.log(F) - (1-probit.y)*np.log(1-F)).sum(axis=0)

-71.39415959767439

In [530]:
r

array([-0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69

In [502]:
from statsmodels.discrete.discrete_model import Probit, ProbitResults

In [500]:
p = Probit(exog=X_mroz, endog=y_mroz)

In [504]:
p_res = p.fit()

Optimization terminated successfully.
         Current function value: 0.603799
         Iterations 5


In [505]:
p_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  753
Model:                         Probit   Df Residuals:                      748
Method:                           MLE   Df Model:                            4
Date:                Wed, 28 Nov 2018   Pseudo R-squ.:                  0.1169
Time:                        15:29:26   Log-Likelihood:                -454.66
converged:                       True   LL-Null:                       -514.87
                                        LLR p-value:                 4.333e-25
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.8803      0.113     -7.759      0.000      -1.103      -0.658
x2            -0.0314      0.007     -4.605      0.000      -0.045      -0.018
x3             0.1584      0.024      6.671      0.000       0.112       0.205
x4            -0.0213      0.005     -4.670      0.000      -0.030      -0.012
const          0.2154      0.417      0.517      0.605      -0.601       1.032
==============================================================================
"""

In [508]:
p.hessian(params=p_res.params)

array([[-1.22238928e+02, -3.31145326e+03, -1.23322188e+03,
        -1.84735246e+03, -9.52369265e+01],
       [-3.31145326e+03, -8.15772572e+05, -2.22804990e+05,
        -3.72245241e+05, -1.83593772e+04],
       [-1.23322188e+03, -2.22804990e+05, -6.54838913e+04,
        -1.08329952e+05, -5.20829413e+03],
       [-1.84735246e+03, -3.72245241e+05, -1.08329952e+05,
        -2.27868162e+05, -8.61498795e+03],
       [-9.52369265e+01, -1.83593772e+04, -5.20829413e+03,
        -8.61498795e+03, -4.27654182e+02]])

In [447]:
probit = ProbitRegression(y_mroz, X_mroz, B=np.array([1, 2, 1, 3, -1]))
H = probit.hessian()

0
0
0


In [457]:
X_mroz[y_mroz==1].shape

(428, 5)

In [409]:
z = f*(probit.y*(f + xb*F)/np.square(F)) + (1-probit.y)*(f + xb*(1-F))/np.square(1-F)

In [416]:
r = probit.X.T@probit.X

In [441]:
(X_mroz.T * z).dot(X_mroz).shape

(5, 5)

In [433]:
outer = X_mroz.T@X_mroz

In [434]:
outer.shape

(5, 5)

In [438]:
z.shape

(753,)

In [435]:
(z.T@(X_mroz)


array([1.38288950e+10, 6.72539041e+11, 2.89002433e+11, 1.72243983e+12,
       1.83243869e+10])

In [417]:
r.shape

(5, 5)

In [341]:
X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])

y = np.array([1, 0, 1, 1, 0])

# Take a guess at initial βs
init_B = np.array([0.1, 0.1, 0.1])

# Create an object with Poisson model values
probit = ProbitRegression(y, X, B=init_B)
B_hat, H = newton_raphson(probit, display=True)

----------
2.552115440931775
9.202796723394986
(5,)
(5,)
(5,)
(5,)
2.552115440931775
9.202796723394986
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -202.70634824  -608.11904471  -527.03650542]
 [ -608.11904471 -2229.76983061 -1378.40316801]
 [ -527.03650542 -1378.40316801 -1743.27459484]]
ERROR  0.4086143234976647
(3,)
2.5521154409317734
9.202796723394977
(5,)
(5,)
(5,)
(5,)
2.5521154409317734
9.202796723394977
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -224.99507381  -674.98522143  -584.98719191]
 [ -674.98522143 -2474.94581193 -1529.96650192]
 [ -584.98719191 -1529.96650192 -1934.95763478]]
ERROR  0.3681356927087527
(3,)
2.552115440931773
9.202796723394965
(5,)
(5,)
(5,)
(5,)
2.552115440931773
9.202796723394965
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -245.07580754  -735.22742263  -637.19709961]
 [ -735.22742263 -2695.83388297 -1666.51549129]
 [ -637.19709961 -1666.51549129 -2107.65194487]]
ERROR  0.3379718226135392
(3,)
2.5521154409317695
9.202796723394956
(5,)
(5,)
(5,)
(5,)
2.5521154409317695
9.202796723

2.552115440931784
9.20279672339496
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -835.59173274 -2506.77519821 -2172.53850511]
 [-2506.77519821 -9191.50906009 -5682.0237826 ]
 [-2172.53850511 -5682.0237826  -7186.08890153]]
ERROR  0.09912582198781728
(3,)
2.5521154409317672
9.202796723394972
(5,)
(5,)
(5,)
(5,)
2.5521154409317672
9.202796723394972
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -840.99875867 -2522.99627601 -2186.59677254]
 [-2522.99627601 -9250.98634535 -5718.79155895]
 [-2186.59677254 -5718.79155895 -7232.58932455]]
ERROR  0.09848851321113017
(3,)
2.5521154409317672
9.202796723394972
(5,)
(5,)
(5,)
(5,)
2.5521154409317672
9.202796723394972
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -846.37102126 -2539.11306377 -2200.56465527]
 [-2539.11306377 -9310.08123382 -5755.32294455]
 [-2200.56465527 -5755.32294455 -7278.79078281]]
ERROR  0.09786336638826576
(3,)
2.5521154409317672
9.202796723394972
(5,)
(5,)
(5,)
(5,)
2.5521154409317672
9.202796723394972
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -851.7091839  -2555.1

ERROR  0.07487982179918973
(3,)
2.5521154409317557
9.20279672339492
(5,)
(5,)
(5,)
(5,)
2.5521154409317557
9.20279672339492
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1110.23985187  -3330.71955561  -2886.62361486]
 [ -3330.71955561 -12212.63837055  -7549.6309927 ]
 [ -2886.62361486  -7549.6309927   -9548.06272607]]
ERROR  0.07460434537119909
(3,)
2.5521154409317557
9.20279672339492
(5,)
(5,)
(5,)
(5,)
2.5521154409317557
9.20279672339492
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1114.30930241  -3342.92790723  -2897.20418627]
 [ -3342.92790723 -12257.40232652  -7577.3032564 ]
 [ -2897.20418627  -7577.3032564   -9583.06000074]]
ERROR  0.07433189077253388
(3,)
2.5521154409317557
9.20279672339492
(5,)
(5,)
(5,)
(5,)
2.5521154409317557
9.20279672339492
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1118.36389135  -3355.09167404  -2907.7461175 ]
 [ -3355.09167404 -12302.00280481  -7604.87446116]
 [ -2907.7461175   -7604.87446116  -9617.92946558]]
ERROR  0.0740624031181163
(3,)
2.5521154409317557
9.20279672339492
(5,

9.202796723395025
(5,)
(5,)
(5,)
(5,)
2.5521154409317677
9.202796723395025
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1392.29132665  -4176.87397996  -3619.9574493 ]
 [ -4176.87397996 -15315.20459319  -9467.58102124]
 [ -3619.9574493   -9467.58102124 -11973.70540922]]
ERROR  0.05949093825984164
(3,)
2.552115440931758
9.202796723395048
(5,)
(5,)
(5,)
(5,)
2.552115440931758
9.202796723395048
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1395.5363847   -4186.60915409  -3628.39460021]
 [ -4186.60915409 -15350.90023166  -9489.64741594]
 [ -3628.39460021  -9489.64741594 -12001.61290839]]
ERROR  0.059352603244132496
(3,)
2.552115440931735
9.202796723395052
(5,)
(5,)
(5,)
(5,)
2.552115440931735
9.202796723395052
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1398.77389697  -4196.3216909   -3636.81213211]
 [ -4196.3216909  -15386.51286663  -9511.66249937]
 [ -3636.81213211  -9511.66249937 -12029.45551391]]
ERROR  0.0592152295187276
(3,)
2.552115440931779
9.202796723395025
(5,)
(5,)
(5,)
(5,)
2.552115440931779
9.20279672339

2.5521154409317677
9.202796723395025
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1531.51302536  -4594.53907608  -3981.93386594]
 [ -4594.53907608 -16846.64327898 -10414.28857246]
 [ -3981.93386594 -10414.28857246 -13171.01201811]]
ERROR  0.054082933662350285
(3,)
2.552115440931758
9.202796723395048
(5,)
(5,)
(5,)
(5,)
2.552115440931758
9.202796723395048
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1534.46309245  -4603.38927734  -3989.60404036]
 [ -4603.38927734 -16879.09401692 -10434.34902864]
 [ -3989.60404036 -10434.34902864 -13196.38259505]]
ERROR  0.053978957044535036
(3,)
2.552115440931779
9.202796723395025
(5,)
(5,)
(5,)
(5,)
2.552115440931779
9.202796723395025
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -1537.40748791  -4612.22246373  -3997.25946857]
 [ -4612.22246373 -16911.48236702 -10454.37091779]
 [ -3997.25946857 -10454.37091779 -13221.70439603]]
ERROR  0.05387557820874278
(3,)
2.5521154409317854
9.202796723395076
(5,)
(5,)
(5,)
(5,)
2.5521154409317854
9.202796723395076
(5,)
(5,)
(5,)
(5,)
(3, 3) (3

...

2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2759.75713321  -8279.27139964  -7175.36854635]
 [ -8279.27139964 -30357.32846534 -18766.34850585]
 [ -7175.36854635 -18766.34850585 -23733.91134563]]
ERROR  0.030013045842629538
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2761.39425777  -8284.1827733   -7179.6250702 ]
 [ -8284.1827733  -30375.33683545 -18777.48095282]
 [ -7179.6250702  -18777.48095282 -23747.9906168 ]]
ERROR  0.0299952522609388
(3,)
2.552115440931751
9.202796723395185
(5,)
(5,)
(5,)
(5,)
2.552115440931751
9.202796723395185
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2763.03041173  -8289.0912352   -7183.87907051]
 [ -8289.0912352  -30393.33452908 -18788.60679979]
 [ -7183.87907051 -18788.60679979 -23762.06154092]]
ERROR  0.02997749029540131
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[

(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2829.29772085  -8487.89316255  -7356.17407421]
 [ -8487.89316255 -31122.27492936 -19239.22450179]
 [ -7356.17407421 -19239.22450179 -24331.96039932]]
ERROR  0.02927536283765164
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2830.89460694  -8492.68382082  -7360.32597804]
 [ -8492.68382082 -31139.84067633 -19250.08332718]
 [ -7360.32597804 -19250.08332718 -24345.69361967]]
ERROR  0.02925884882843377
(3,)
2.5521154409318276
9.202796723395029
(5,)
(5,)
(5,)
(5,)
2.5521154409318276
9.202796723395029
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2832.49059223  -8497.4717767   -7364.47553981]
 [ -8497.4717767  -31157.39651458 -19260.9360272 ]
 [ -7364.47553981 -19260.9360272  -24359.41909322]]
ERROR  0.029242362739253736
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2834.08567826  -8502.25703479  

9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2915.8273296   -8747.48198881  -7581.15105697]
 [ -8747.48198881 -32074.10062563 -19827.6258413 ]
 [ -7581.15105697 -19827.6258413  -25076.11503458]]
ERROR  0.02840659202030338
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2917.37682676  -8752.13048029  -7585.17974958]
 [ -8752.13048029 -32091.14509439 -19838.16242199]
 [ -7585.17974958 -19838.16242199 -25089.44071016]]
ERROR  0.02839150451659833
(3,)
2.552115440931751
9.202796723395185
(5,)
(5,)
(5,)
(5,)
2.552115440931751
9.202796723395185
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2918.92550094  -8756.77650283  -7589.20630245]
 [ -8756.77650283 -32108.18051039 -19848.69340642]
 [ -7589.20630245 -19848.69340642 -25102.75930812]]
ERROR  0.028376441031767285
(3,)
2.552115440931751
9.202796723395185
(5,)
(5,)
(5,)
(5,)
2.552115440931751
9.202796723395185
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -2920.47335346  

2.5521154409318276
9.202796723395029
(5,)
(5,)
(5,)
(5,)
2.5521154409318276
9.202796723395029
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -3001.36687143  -9004.1006143   -7803.55386572]
 [ -9004.1006143  -33015.03558576 -20409.29472574]
 [ -7803.55386572 -20409.29472574 -25811.75509432]]
ERROR  0.02759699860154541
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -3002.87220762  -9008.61662287  -7807.46773982]
 [ -9008.61662287 -33031.59428384 -20419.53101183]
 [ -7807.46773982 -20419.53101183 -25824.70098555]]
ERROR  0.027583164259681325
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
(3, 3) (3,)
[[ -3004.37678919  -9013.13036756  -7811.37965189]
 [ -9013.13036756 -33048.14468106 -20429.76216647]
 [ -7811.37965189 -20429.76216647 -25837.64038701]]
ERROR  0.02756935070586053
(3,)
2.552115440931762
9.202796723395078
(5,)
(5,)
(5,)
(5,)
2.552115440

/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/Users/Lraymond/Python/patents/patents_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()


In [342]:
B_hat

array([51.655107,  0.1     ,  0.1     ])

In [300]:
z[np.isinf(z)] = 0

----------
Log Liklihood : -54.7061685732084
363.2037844830384
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 2.  1. -1.  1.]
B NEW  [ 2.  1. -1.  1.]
Log Liklihood : -54.706168548908806
363.2037844780075
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 2.  1. -1.  1.]
B NEW  [ 1.99999999  1.         -1.          1.        ]
Log Liklihood : -54.706168524284976
363.2037844729764
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999999  1.         -1.          1.        ]
B NEW  [ 1.99999999  1.         -1.00000001  1.        ]
Log Liklihood : -54.70616849998538
363.2037844679454
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999999  1.         -1.00000001  1.        ]
B NEW  [ 1.99999998  1.         -1.00000001  1.        ]
Log Liklihood : -54.70616847533779
363.2037844629144
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999998  1.

Log Liklihood : -54.70616678743107
363.20378411577195
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999967  0.99999995 -1.00000016  1.        ]
B NEW  [ 1.99999966  0.99999995 -1.00000016  1.        ]
Log Liklihood : -54.70616676313125
363.20378411074097
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999966  0.99999995 -1.00000016  1.        ]
B NEW  [ 1.99999966  0.99999995 -1.00000016  1.        ]
Log Liklihood : -54.70616673848377
363.20378410570976
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999966  0.99999995 -1.00000016  1.        ]
B NEW  [ 1.99999965  0.99999994 -1.00000016  1.        ]
Log Liklihood : -54.706166714183894
363.2037841006789
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999965  0.99999994 -1.00000016  1.        ]
B NEW  [ 1.99999965  0.99999994 -1.00000016  1.        ]
Log Liklihood : -54.70616668957683
363.20378409564785
0


Log Liklihood : -54.7061658577544
363.20378392459213
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999949  0.99999992 -1.00000024  1.        ]
B NEW  [ 1.99999949  0.99999992 -1.00000024  1.        ]
Log Liklihood : -54.70616583343832
363.20378391956115
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999949  0.99999992 -1.00000024  1.        ]
B NEW  [ 1.99999948  0.99999992 -1.00000024  1.        ]
Log Liklihood : -54.706165808830235
363.20378391453016
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999948  0.99999992 -1.00000024  1.        ]
B NEW  [ 1.99999948  0.99999992 -1.00000024  1.        ]
Log Liklihood : -54.70616578452808
363.2037839094992
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.4638958368304884e-11
B OLD  [ 1.99999948  0.99999992 -1.00000024  1.        ]
B NEW  [ 1.99999947  0.99999992 -1.00000025  1.        ]
Log Liklihood : -54.70616575988333
363.2037839044681
0
0


Log Liklihood : -54.70616407197565
363.203783557326
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999916  0.99999987 -1.00000039  1.00000001]
B NEW  [ 1.99999915  0.99999987 -1.0000004   1.00000001]
Log Liklihood : -54.70616404763837
363.203783552295
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999915  0.99999987 -1.0000004   1.00000001]
B NEW  [ 1.99999915  0.99999986 -1.0000004   1.00000001]
Log Liklihood : -54.70616402333701
363.2037835472639
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999915  0.99999986 -1.0000004   1.00000001]
B NEW  [ 1.99999914  0.99999986 -1.0000004   1.00000001]
Log Liklihood : -54.70616399872989
363.2037835422328
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999914  0.99999986 -1.0000004   1.00000001]
B NEW  [ 1.99999914  0.99999986 -1.0000004   1.00000001]
Log Liklihood : -54.70616397441631
363.20378353720184
0
0
0
0
0
0


Log Liklihood : -54.706162653324895
363.20378326552566
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999889  0.99999982 -1.00000052  1.00000001]
B NEW  [ 1.99999889  0.99999982 -1.00000052  1.00000001]
Log Liklihood : -54.70616262867969
363.20378326049456
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999889  0.99999982 -1.00000052  1.00000001]
B NEW  [ 1.99999888  0.99999982 -1.00000052  1.00000001]
Log Liklihood : -54.706162604363385
363.2037832554636
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999888  0.99999982 -1.00000052  1.00000001]
B NEW  [ 1.99999888  0.99999982 -1.00000052  1.00000001]
Log Liklihood : -54.70616257975621
363.20378325043254
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999888  0.99999982 -1.00000052  1.00000001]
B NEW  [ 1.99999887  0.99999982 -1.00000053  1.00000001]
Log Liklihood : -54.70616255545741
363.2037832454013
0
0
0


...

0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999612  0.99999938 -1.00000181  1.00000003]
B NEW  [ 1.99999611  0.99999938 -1.00000182  1.00000003]
Log Liklihood : -54.70614780444737
363.2037802116889
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999611  0.99999938 -1.00000182  1.00000003]
B NEW  [ 1.99999611  0.99999938 -1.00000182  1.00000003]
Log Liklihood : -54.70614778014567
363.2037802066578
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999611  0.99999938 -1.00000182  1.00000003]
B NEW  [ 1.9999961   0.99999938 -1.00000182  1.00000003]
Log Liklihood : -54.70614775553895
363.2037802016267
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.9999961   0.99999938 -1.00000182  1.00000003]
B NEW  [ 1.9999961   0.99999938 -1.00000182  1.00000003]
Log Liklihood : -54.70614773123992
363.2037801965957
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-

363.2037799752308
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.9999959   0.99999935 -1.00000192  1.00000003]
B NEW  [ 1.99999589  0.99999935 -1.00000192  1.00000003]
Log Liklihood : -54.70614663046791
363.20377997019966
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999589  0.99999935 -1.00000192  1.00000003]
B NEW  [ 1.99999589  0.99999935 -1.00000192  1.00000003]
Log Liklihood : -54.70614660582015
363.2037799651687
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999589  0.99999935 -1.00000192  1.00000003]
B NEW  [ 1.99999588  0.99999935 -1.00000192  1.00000003]
Log Liklihood : -54.70614658152044
363.2037799601378
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999588  0.99999935 -1.00000192  1.00000003]
B NEW  [ 1.99999588  0.99999934 -1.00000193  1.00000003]
Log Liklihood : -54.706146556911676
363.2037799551066
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR 

Log Liklihood : -54.70614540719532
363.2037797186485
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999566  0.99999931 -1.00000203  1.00000003]
B NEW  [ 1.99999566  0.99999931 -1.00000203  1.00000003]
Log Liklihood : -54.706145382893226
363.2037797136177
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999566  0.99999931 -1.00000203  1.00000003]
B NEW  [ 1.99999565  0.99999931 -1.00000203  1.00000003]
Log Liklihood : -54.706145358286165
363.20377970858647
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999565  0.99999931 -1.00000203  1.00000003]
B NEW  [ 1.99999565  0.99999931 -1.00000203  1.00000003]
Log Liklihood : -54.70614533398668
363.2037797035555
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999565  0.99999931 -1.00000203  1.00000003]
B NEW  [ 1.99999565  0.99999931 -1.00000203  1.00000003]
Log Liklihood : -54.7061453093379
363.2037796985247
0
0
0
0
0

Log Liklihood : -54.706144159660425
363.2037794620665
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999543  0.99999927 -1.00000214  1.00000003]
B NEW  [ 1.99999543  0.99999927 -1.00000214  1.00000003]
Log Liklihood : -54.70614413505018
363.2037794570356
0
0
0
0
0
0
Shapes H, G:  (4, 4) (4,)
ERROR  3.463873632369996e-11
B OLD  [ 1.99999543  0.99999927 -1.00000214  1.00000003]
B NEW  [ 1.99999542  0.99999927 -1.00000214  1.00000003]
Number of iterations: 1000
B_hat =  [ 1.99999542  0.99999927 -1.00000214  1.00000003]
Log Lik =  -54.70614411071256
363.2037794520046
0
0
0


ValueError: too many values to unpack (expected 2)

In [174]:
B_hat

array([0.1, 0.1, 0.1, 0.1, 0.1])

In [348]:
probit = ProbitRegression(y, X, B=init_B)
probit.hessian()

ValueError: shapes (5,3) and (4,) not aligned: 3 (dim 1) != 4 (dim 0)

In [59]:
f*y*(f/F) - (1-y)*f/(1-F)

array([ 0.10647944, -1.14107777,  0.12862512,  0.10647944, -1.14107777])

In [34]:
# Use newton_raphson to find the MLE


----------
()
()


ValueError: Scalar operands are not allowed, use '*' instead